In [ ]:
# i have 3 folders, train test and sample
# write code to import training csv and testing csv

import pandas as pd

# Import the training data
train_data = pd.read_csv('train/train.csv')

# Import the testing data
test_data = pd.read_csv('test/test.csv')


print(train_data.columns)

In [103]:
# now in train data , 
from skimpy import skim
from sklearn.preprocessing import StandardScaler


df = train_data.drop('id', axis = 1)

df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

# Extract relevant features from the datetime columns
df['pickup_year'] = df['pickup_datetime'].dt.year
df['pickup_month'] = df['pickup_datetime'].dt.month
df['pickup_day'] = df['pickup_datetime'].dt.day
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_minute'] = df['pickup_datetime'].dt.minute

# df['dropoff_year'] = df['dropoff_datetime'].dt.year
# df['dropoff_month'] = df['dropoff_datetime'].dt.month
# df['dropoff_day'] = df['dropoff_datetime'].dt.day
# df['dropoff_hour'] = df['dropoff_datetime'].dt.hour
# df['dropoff_minute'] = df['dropoff_datetime'].dt.minute

df = df.drop(['pickup_year'], axis=1)

# Drop the original datetime columns
df.drop(['pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

# Scale the data using StandardScaler
# scaler = StandardScaler()
# df[['pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour', 'pickup_minute',
#      'dropoff_year', 'dropoff_month', 'dropoff_day', 'dropoff_hour', 'dropoff_minute']] = scaler.fit_transform(df[['pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour', 'pickup_minute',
#                                                                                                                  'dropoff_year', 'dropoff_month', 'dropoff_day', 'dropoff_hour', 'dropoff_minute']])



In [104]:
# now in train data , 
from skimpy import skim
from sklearn.preprocessing import StandardScaler


df_test = test_data.drop('id', axis = 1)

df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])
# df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

# Extract relevant features from the datetime columns
df_test['pickup_year'] = df_test['pickup_datetime'].dt.year
df_test['pickup_month'] = df_test['pickup_datetime'].dt.month
df_test['pickup_day'] = df_test['pickup_datetime'].dt.day
df_test['pickup_hour'] = df_test['pickup_datetime'].dt.hour
df_test['pickup_minute'] = df_test['pickup_datetime'].dt.minute

df_test = df_test.drop(['pickup_year'], axis=1)

# Drop the original datetime columns
df_test.drop(['pickup_datetime'], axis=1, inplace=True)

In [105]:
import numpy as np

# Define a function to perform cyclical transformation
def cyclical_transform(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    df.drop(col, axis=1, inplace=True)

# Apply cyclical transformation to date, month, minute, and hour
cyclical_transform(df, 'pickup_day', 31)
cyclical_transform(df, 'pickup_month', 12)
cyclical_transform(df, 'pickup_minute', 60)
cyclical_transform(df, 'pickup_hour', 24)

# cyclical_transform(df, 'dropoff_day', 31)
# cyclical_transform(df, 'dropoff_month', 12)
# cyclical_transform(df, 'dropoff_minute', 60)
# cyclical_transform(df, 'dropoff_hour', 24)



In [106]:
import numpy as np

# Define a function to perform cyclical transformation
def cyclical_transform(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    df.drop(col, axis=1, inplace=True)

# Apply cyclical transformation to date, month, minute, and hour
cyclical_transform(df_test, 'pickup_day', 31)
cyclical_transform(df_test, 'pickup_month', 12)
cyclical_transform(df_test, 'pickup_minute', 60)
cyclical_transform(df_test, 'pickup_hour', 24)


In [ ]:
df['passenger_count_log'] = np.sqrt(df['passenger_count'] + 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','vendor_id']] = scaler.fit_transform(df[['passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','vendor_id']])
df = df.drop(['passenger_count_log'],axis = 1)

print(df)

In [ ]:
df_test['passenger_count_log'] = np.sqrt(df_test['passenger_count'] + 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_test[['passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','vendor_id']] = scaler.fit_transform(df_test[['passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','vendor_id']])
df_test = df_test.drop(['passenger_count_log'],axis = 1)

print(df_test)

In [ ]:
df = pd.get_dummies(df, columns=['store_and_fwd_flag'])
df_test = pd.get_dummies(df_test, columns=['store_and_fwd_flag'])

skim(df_test)

In [110]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X = df.drop('trip_duration', axis=1)
y = df['trip_duration']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Create a Random Forest Regressor model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
rf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# You can also use other metrics such as mean absolute error, R-squared, etc.

Mean Squared Error: 13891572.42


In [ ]:
# Get the feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to store the feature importances
feature_importances_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importances_df = feature_importances_df.sort_values(by='importance', ascending=False)

# Print the top 10 most important features
print(feature_importances_df.head(10))

import joblib

# Save the trained Random Forest Regressor model to a file
joblib.dump(rf, 'random_forest_regressor.pkl')




In [ ]:
# skim(df)

print(X_test.columns)
print(df_test.columns)


y_test_pred = rf.predict(df_test)

In [ ]:
## Uncomment this line if you want to try Gradient Boosting Regressor

# from sklearn.ensemble import GradientBoostingRegressor

# # Create a Gradient Boosting Regressor model
# gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# # Train the model on the training data
# gbr.fit(X_train, y_train)

# # Make predictions on the testing data
# y_pred = gbr.predict(X_test)

# # Evaluate the model using mean squared error
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error: {mse:.2f}')

# # Get the feature importances
# feature_importances = gbr.feature_importances_

# # Create a DataFrame to store the feature importances
# feature_importances_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})

# # Sort the DataFrame by importance in descending order
# feature_importances_df = feature_importances_df.sort_values(by='importance', ascending=False)

# # Print the top 10 most important features
# print(feature_importances_df.head(10))

# # Create a DataFrame with the predictions
# predictions_df = pd.DataFrame({'trip_duration': y_pred})

# # Save the predictions to a CSV file
# predictions_df.to_csv('trip_duration_predictions_gbr.csv', index=False)

In [117]:
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame({'trip_duration': y_test_pred})

# Save the predictions to a CSV file
predictions_df.to_csv('trip_duration_predictions_3.csv', index=False)

In [ ]:
pd.read_csv('./test/test - Copy.csv')